In [1]:
import sys
import os 
from pathlib import Path
import pandas as pd
from bs4 import BeautifulSoup 
import requests
import lxml
import json

project_path = Path(os.path.dirname(os.path.realpath("__file__"))).parent

from scripts.utils import log_df

In [61]:
constellation_path = project_path/'raw_data'/'constellations.csv'
constellation_js_path = project_path/'data'/'constellations.js'

solar_system_path = project_path/'raw_data'/'solar_system.csv'


# Constellations

## get constellation data

In [3]:
wikiurl="https://en.wikipedia.org/wiki/IAU_designated_constellations"
# table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [4]:
soup = BeautifulSoup(response.text, 'html.parser')
table=soup.find('table',{'class':"wikitable"})

In [5]:
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])
df.columns = ['Constellation', 'Abbreviations IAU',
              'Abbreviations NASA', 'Genitive',
              'Origin', 'Meaning', 'Brightest star']
log_df(df)

(88, 7)


,Constellation,Abbreviations IAU,Abbreviations NASA,Genitive,Origin,Meaning,Brightest star
0,Andromeda /ænˈdrɒmɪdə/[7],And,Andr,Andromedae /ænˈdrɒmɪdiː/,ancient (Ptolemy),Andromeda (The chained maiden or princess),Alpheratz
1,Antlia /ˈæntliə/[7],Ant,Antl,Antliae /ˈæntliiː/,"1763, Lacaille",air pump,α Antliae
2,Apus /ˈeɪpəs/[8],Aps,Apus,Apodis /ˈæpoʊdɪs/[8],"1603, Uranometria, created by Keyser and de Ho...",Bird-of-paradise/Exotic Bird/Extraordinary Bird,α Apodis
3,Aquarius /əˈkwɛəriəs/[7],Aqr,Aqar,Aquarii /əˈkwɛəriaɪ/,ancient (Ptolemy),water-bearer,Sadalsuud
4,Aquila /ˈækwɪlə/[7],Aql,Aqil,Aquilae /ˈækwɪliː/,ancient (Ptolemy),eagle,Altair


In [6]:
df[['name', 'b']] = df['Constellation'].str.split(' /', expand=True)
del df['b']
log_df(df)

(88, 8)


,Constellation,Abbreviations IAU,Abbreviations NASA,Genitive,Origin,Meaning,Brightest star,name
0,Andromeda /ænˈdrɒmɪdə/[7],And,Andr,Andromedae /ænˈdrɒmɪdiː/,ancient (Ptolemy),Andromeda (The chained maiden or princess),Alpheratz,Andromeda
1,Antlia /ˈæntliə/[7],Ant,Antl,Antliae /ˈæntliiː/,"1763, Lacaille",air pump,α Antliae,Antlia
2,Apus /ˈeɪpəs/[8],Aps,Apus,Apodis /ˈæpoʊdɪs/[8],"1603, Uranometria, created by Keyser and de Ho...",Bird-of-paradise/Exotic Bird/Extraordinary Bird,α Apodis,Apus
3,Aquarius /əˈkwɛəriəs/[7],Aqr,Aqar,Aquarii /əˈkwɛəriaɪ/,ancient (Ptolemy),water-bearer,Sadalsuud,Aquarius
4,Aquila /ˈækwɪlə/[7],Aql,Aqil,Aquilae /ˈækwɪliː/,ancient (Ptolemy),eagle,Altair,Aquila


In [7]:
 df.to_csv(constellation_path, index=False)

In [8]:
object_dict = {}
for index, row in df.iterrows():
    object_dict[row['Abbreviations IAU']] = row['name']

json_object = json.dumps(object_dict)
 
with open(constellation_js_path, "w") as outfile:
    outfile.write(json_object)

# Planets

## get planet data

In [13]:
url = 'https://en.wikipedia.org/wiki/Apparent_magnitude'
response=requests.get(url)
print(response.status_code)

200


In [39]:
soup = BeautifulSoup(response.text, 'html.parser')
tables=soup.findAll('table')

In [57]:
df = pd.read_html(str(tables))
df = pd.DataFrame(df[5])
df = df.fillna('')

log_df(df)

(125, 4)


,Apparent magnitude (V),Object,Seen from...,Notes
0,−67.57,gamma-ray burst GRB 080319B,seen from 1 AU away,would be over 2×1016 (20 quadrillion) times as...
1,−41.39,star Cygnus OB2-12,seen from 1 AU away,
2,−40.67,star M33-013406.63,seen from 1 AU away,
3,–40.17,star Eta Carinae A,seen from 1 AU away,
4,−40.07,star Zeta1 Scorpii,seen from 1 AU away,


In [58]:
planets_df = df[df['Object'].str.startswith('planet')]
planets_df = planets_df[planets_df['Notes'].str.startswith('mean')]

planets_df

,Apparent magnitude (V),Object,Seen from...,Notes
34,−4.14,planet Venus,seen from Earth,mean brightness[32]
45,−2.20,planet Jupiter,seen from Earth,mean brightness[32]
56,+0.23,planet Mercury,seen from Earth,mean brightness[32]
58,+0.46,planet Saturn,seen from Earth,mean brightness[32]
59,+0.71,planet Mars,seen from Earth,mean brightness[32]
74,+5.68,planet Uranus,seen from Earth,mean brightness[32]
85,+7.78,planet Neptune,seen from Earth,mean brightness[32]


In [67]:
moon_df = df[df['Object'] == 'Moon']
moon_df = moon_df[moon_df['Seen from...'].str.startswith('lit by earthlight,')]

moon_df

,Apparent magnitude (V),Object,Seen from...,Notes
37,−3.69,Moon,"lit by earthlight, reflecting earthshine seen ...",


In [68]:
combine_df = pd.concat([moon_df, planets_df])

combine_df

,Apparent magnitude (V),Object,Seen from...,Notes
37,−3.69,Moon,"lit by earthlight, reflecting earthshine seen ...",
34,−4.14,planet Venus,seen from Earth,mean brightness[32]
45,−2.20,planet Jupiter,seen from Earth,mean brightness[32]
56,+0.23,planet Mercury,seen from Earth,mean brightness[32]
58,+0.46,planet Saturn,seen from Earth,mean brightness[32]
59,+0.71,planet Mars,seen from Earth,mean brightness[32]
74,+5.68,planet Uranus,seen from Earth,mean brightness[32]
85,+7.78,planet Neptune,seen from Earth,mean brightness[32]


In [70]:
combine_df.to_csv(solar_system_path, index=False)